# Spam or ham?

In the spam literature, an email that is **not** spam is called _ham_. 

Your task is to develop a naïve Bayes classifier to determine whether a given email is spam or ham.

The following cell loads your training data set (called _training set_).

In [1]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam (`1.`) or ham (`0.`). The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value `1.` if the keyword appears in the message and `0.` otherwise.

## The model:  naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* **$C = 1$ for spam messages **
* **$C = 0$ for ham messages. **


The classifier builds a model for the probability $P(C=c\ |\ message)$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate
$\require{color}$
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \  \textcolor{blue}{P(C=c\ |\ message)}.
\end{equation}
Using Bayes' rule we can write

\begin{equation}
P(C=c\ |\ message) = \frac{P(message\ |\ C=c)P(C=c)}{P(message\ |\ C=1)P(C=1) + P(message\ |\ C=0)P(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
\textcolor{blue}{P(C=c\ |\ message)} \propto \textcolor{orange}{P(message\ |\ C=c)}\textcolor{green}{P(C=c)},
\end{equation}

where $\propto$ means "proportional to". The class priors $\textcolor{green}{P(C=c)}$ can be computed directly (you will do so in exercise A) but we need to further simplify $\textcolor{orange}{P(message\ |\ C=c)}$.


### Choice of the event model: _Multinomial_ naïve Bayes

Different naïve Bayes models differ in their distributional assumptions about $\textcolor{orange}{P(message\ |\ C=c)}$. We represent a message using a **binary** [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as a set of $k$ keywords, that is, $message = (w_1, ..., w_k)$, where $w_i = 1$ if the  keyword $w_i$ appears in the message and $w_i = 0$ otherwise.

We assume that the $P(w_1, ..., w_k |\ C=c)$ follows a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) for each class. Note that this event model is slightly different from the one used in the lecture, where we assumed a [multivariate Bernoulli event model](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Parameter_estimation_and_event_models). Intuitively, the multinomial distribution assumes that the words of a message were "drawn" independently from a bag of $k$ different words. Depending on the class membership $c$, each keyword $w$ has a probability $\theta_{c, w}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy,... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment,... $\}$, if the training data was mostly gathered from emails of researchers.

Under these assumptions, the likelihood of a message, given that it belongs to class $c$, is then proportional to
\begin{equation}
\textcolor{orange}{P(message\ |\ C=c)} \propto \prod_{i = 1}^k  (\textcolor{brown}{\theta_{c, w_i}})^{w_i}.
\end{equation}


The parameters $\textcolor{brown}{\theta_{c, w}}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$, that is,
\begin{equation}
\textcolor{brown}{\theta_{c, w}} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}
where $n_{c, w}$ is the number of times the keyword $w$ appears in messages of class $c$ in the training set and $n_{c}$ is the total count of keywords for all messages of class $c$, that is, $n_{c} = \sum_w n_{c, w}$.



We are now finally able to rewrite the *maximum a posteriori* estimate in a form that is easy to compute:
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ [ \textcolor{green}{P(C=c)}   \prod_{i = 1}^k  (\textcolor{brown}{\theta_{c, w_i}})^{w_i}].
\end{equation}


#### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions, that is,
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ log( \textcolor{green}{P(C=c)}   \prod_{i = 1}^k  (\textcolor{brown}{\theta_{c, w_i}})^{w_i} ) \\
 = \underset{c \in \{0,1\}}{argmax} \ [ log( \textcolor{green}{P(C=c)}) + \sum_{i = 1}^k w_i \ log(\textcolor{brown}{\theta_{c, w_i}}) ].
\end{equation}

## Part A: Estimate class priors (20 marks)

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns a numpy array containing the **logarithm** of the empirical class priors $\textcolor{green}{P(C=c)}$ for $c \in \{0, 1\}$.

Use the function **np.log()** (the natural log) to compute logarithms throughout this notebook. Avoid using all other bases for logarithm computations.

In [2]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
    log(P(C=0)) and log(P(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
    ### YOUR CODE HERE...
    #store chance of belonging to either class
    log_class_priors = np.array([0.,0.])
    #for every sample
    for sample in data:
        #increment number of class 0 or class 1 depending on what the sample is (1 = spam, 0 = ham)
        log_class_priors[(int)(sample[0])] += 1
    
    #calculate proportion by dividing by how many classes there are, and take logarithm of probability
    for c in range(len(log_class_priors)):
        log_class_priors[c] = np.log(log_class_priors[c] / len(data))  
    
    return log_class_priors

In [3]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam)
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)

# Check wehther the values are both negative (the logarithm of a probability 0 < p < 1 should be negative).
assert(np.all(log_class_priors < 0))


result [-0.48939034 -0.94933059]


## Part B: Estimate class-conditional likelihoods (40 marks)
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns **the logarithm** of the empirical class-conditional likelihoods $log(\textcolor{brown}{\theta_{c, w_i}})$ for all words $w_i$ and both classes ($c \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial event model and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [4]:
def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features (words), calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    ### YOUR CODE HERE...
    #store the number of features (i.e. keywords), theta (chance keyword is in sample of class)
    # and count of total keywords in a class
    n_features = len(data[0])-1 
    theta = np.zeros((2,n_features))
    class_keyword_counts = [0,0]
    
    #for each sample in the data
    for sample in data:
        #for every keyword in that sample
        for keyword in range(1,len(sample)):
            #if keyword is in sample, increment count of that keyword in sample, and total keywords in class
            if((int)(sample[keyword])):
                theta[(int)(sample[0])][keyword-1] += 1
                class_keyword_counts[(int)(sample[0])] += 1
    
    #for each class
    for class_ in range(len(theta)):
        #for each keyword in that class
        for keyword in range(len(theta[class_])):
            #calculate probability of that keyword belonging to that class
            #use laplace smoothing if alpha > 0
            if alpha > 0:
                theta[class_][keyword] = np.log((theta[class_][keyword] + alpha) / (class_keyword_counts[class_] + (n_features*alpha)))
            else:
                theta[class_][keyword] = np.log((theta[class_][keyword]) / (class_keyword_counts[class_]))
    
    return theta


In [5]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)
print(log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)


[[-3.83854747 -4.40050244 -3.37557787 -7.81822913 -3.56973388 -4.35249322
  -5.94642695 -4.79780424 -4.52239226 -3.88640349 -4.87379015 -2.9354272
  -4.11692715 -5.1791718  -6.02646966 -4.40050244 -4.33698904 -4.11692715
  -2.67073465 -5.94642695 -3.16426878 -6.71961684 -5.67816296 -5.94642695
  -3.06463893 -3.36388183 -3.41150988 -3.94702811 -4.11692715 -4.0005168
  -4.38424192 -4.6401753  -4.2918686  -4.61955601 -3.95749941 -3.87664732
  -3.34089231 -6.02646966 -4.26288106 -4.40050244 -5.1791718  -4.18064297
  -4.33698904 -4.38424192 -3.31287927 -3.85741596 -6.02646966 -4.90045839
  -3.78398849 -2.68243069 -4.01156664 -3.32959276 -4.35249322 -4.54108439]
 [-3.54302758 -3.73766962 -3.08672279 -6.58755001 -2.9961259  -3.4520558
  -3.47086513 -3.59181774 -3.77986997 -3.31852441 -3.77986997 -3.04039872
  -3.70514643 -4.64163987 -4.52612698 -3.13529746 -3.39766173 -3.62080528
  -2.68125768 -4.05612335 -2.78088753 -5.58902118 -3.68927308 -3.48360416
  -6.04855351 -6.33623559 -7.84031298 -6

## Part  C: Classify e-mails (30 marks)

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

Use your `predict()` function to classify the messages of your **training data set** `training_spam`. Compute the accuracy of your algorithm _in the training set_ by comparing your predictions to the true class values. Accuracy is simply defined as the proportion of true predictions made by your classifier. Store the accuracy of your naïve Bayes algorithm in a variable called `training_set_accuracy`.

In [13]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    ### YOUR CODE HERE...
    class_predictions = np.zeros(new_data.shape[0]) 
    
    for sample in range(0,len(new_data)):
        sample_prediction = [log_class_priors[0],log_class_priors[1]] 
        for feature in range(len(new_data[sample])):
            sample_prediction[1] += new_data[sample,feature] * log_class_conditional_likelihoods[1][feature]
            sample_prediction[0] += new_data[sample,feature] * log_class_conditional_likelihoods[0][feature]
            
        class_predictions[sample] = sample_prediction.index(max(sample_prediction))
    
    class_predictions = np.array(class_predictions)
    return class_predictions

def accuracy(y_predictions, y_true):
    """
    Calculate the accuracy.
    
    :param y_predictions: a one-dimensional numpy array of predicted classes (0s and 1s).
    :param y_true: a one-dimensional numpy array of true classes (0s and 1s).
    
    :return acc: a float between 0 and 1 
    """
    ### YOUR CODE HERE...
    acc = 0
    for y in range(0,len(y_true)):
        
        if(y_predictions[y] == y_true[y]):
            acc += 1
    
    acc = acc / len(y_true)
    
    return acc

training_set_accuracy = accuracy(predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods),training_spam[:,0])
print("training_set_accuracy:",training_set_accuracy)

training_set_accuracy: 0.892


In [14]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))
       
# Check accuracy function
true_classes = training_spam[:, 0]
training_set_accuracy = accuracy(class_predictions, true_classes)
assert(isinstance(training_set_accuracy, float))
assert(0 <= training_set_accuracy <= 1)


## Part D: Classifying previously unseen data (10 marks).
The following cell loads a new set of 500 messages.

In [15]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",")
print("Shape of the testing spam data set:", testing_spam.shape)
testing_spam

Shape of the testing spam data set: (500, 55)


array([[1., 0., 0., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Use the naïve Bayes algorithm that you trained on `training_spam` in order to classify all messages in the `testing_spam` data set. Store the resulting accuracy in a variable called `testing_set_accuracy`.

In [16]:
log_class_priors = estimate_log_class_priors(training_spam)
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)
class_predictions = predict(testing_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)
true_classes = testing_spam[:, 0]
testing_set_accuracy = accuracy(class_predictions, true_classes)
print("Testing_set_accuracy: ",testing_set_accuracy)


#testing_set_accuracy = accuracy(predict(testing_spam[:,1:],log_class_priors, log_class_conditional_likelihoods),testing_spam)

Testing_set_accuracy:  0.838


In [17]:
# This is a test cell. Do not delete or change.

## Part E: Discussion (not marked).


In 3 sentences or less: Compare the accuracy of your classifier on the training set and on the test set. Are they different? If yes, how do you explain the difference?

The classifier seems to have a higher accuracy on the training set than the testing set (89.2% vs 83.8%). I expect the training set has higher accuracy since it is what the model is based off. However, due to the value of certain words on if an email is spam or ham, we may have achieved a 'good' (i.e. representative) sample, which included words which heavily indicated spam or ham, hence only the small difference in accuracy.

